In [ ]:
# STEP 1: Install required libraries
!pip install transformers datasets arabert nltk gradio openpyxl
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 11.5 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186456 sha256=f8678ed63561bbb9f9c4d02871c74911a8c17e2a97afe3a33e725d3d7df76ae8
  Stored in directory: /root/.cache/pip/wheels/bb/f1/26/f9002669ef6ad80a3c9f1b22880b35d9b4c6650011acee0523
Successfully built emoji
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
# STEP 2: Import libraries
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import torch
import nltk
import gradio as gr

In [ ]:
# STEP 3: Load dataset (Excel)
df = pd.read_csv("/content/balanced_dataset.csv")
df = df[['tweet','category']]   # Ensure only needed columns

df.head()

,tweet,category
0,عجينة ملفات تعريف الارتباط مثل تأكلني الخام ...,1
1,شعر شعر جديد ونظيف,0
2,طارق لم أكن أريد التفكير في الأمر,0
3,أتمنى حقًا أن أكون هناك لإعطائك بعضًا من ...,0
4,نحن نعلم أنه ليس لدينا ملكة,0


In [ ]:
# STEP 4: Preprocess Arabic text (AraBERT Preprocessor)
from arabert.preprocess import ArabertPreprocessor
model_name = "aubmindlab/bert-base-arabertv02"
arabert_prep = ArabertPreprocessor(model_name=model_name)

df['tweet'] = df['tweet'].astype(str).apply(lambda x: arabert_prep.preprocess(x))

In [ ]:
# STEP 5: Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['tweet'], df['category'], test_size=0.2, random_state=42
)

# Convert to HuggingFace dataset
train_df = pd.DataFrame({'text':train_texts, 'label':train_labels})
val_df = pd.DataFrame({'text':val_texts, 'label':val_labels})

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df)
})

In [ ]:
# STEP 6: Load tokenizer & tokenize dataset
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.remove_columns(["text", "__index_level_0__"])
dataset.set_format("torch")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/1286 [00:00<?, ? examples/s]

Map:   0%|          | 0/322 [00:00<?, ? examples/s]

In [ ]:
# STEP 7: Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# STEP 8: Define metrics
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels),
        "f1": f1.compute(predictions=preds, references=labels, average="weighted")
    }

In [ ]:
# STEP 9: Training setup
training_args = TrainingArguments(
    output_dir="./results",
    do_eval=True,
    eval_steps=500,
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none"
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2297803256.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
df['category'].value_counts()

,count
category,
0,840
1,768


In [ ]:
# STEP 10: Train model
trainer.train()

Step,Training Loss
500,0.124000


TrainOutput(global_step=567, training_loss=0.11308361151020783, metrics={'train_runtime': 220.5757, 'train_samples_per_second': 40.811, 'train_steps_per_second': 2.571, 'total_flos': 592131430087680.0, 'train_loss': 0.11308361151020783, 'epoch': 7.0})

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

preds_output = trainer.predict(dataset["validation"])
y_true = preds_output.label_ids
y_pred = preds_output.predictions.argmax(-1)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, digits=3))

[[162   7]
 [ 19 134]]
              precision    recall  f1-score   support

           0      0.895     0.959     0.926       169
           1      0.950     0.876     0.912       153

    accuracy                          0.919       322
   macro avg      0.923     0.917     0.919       322
weighted avg      0.921     0.919     0.919       322



In [ ]:
# Evaluate on validation set
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.3655477464199066, 'eval_accuracy': {'accuracy': 0.9192546583850931}, 'eval_f1': {'f1': 0.9189910001267588}, 'eval_runtime': 2.7349, 'eval_samples_per_second': 117.736, 'eval_steps_per_second': 7.678, 'epoch': 7.0}


In [ ]:
# STEP 11: Save model
trainer.save_model("arabert-suicidal-detector")

In [ ]:
!pip install gradio torch transformers deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from deep_translator import GoogleTranslator

# Load trained model
model_path = "arabert-suicidal-detector"  # folder where you saved the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

labels = ["Non-Suicidal", "Suicidal"]

def classify_tweet(text):
    # 🔹 Step 1: Translate Arabic → English
    try:
        english_text = GoogleTranslator(source='auto', target='en').translate(text)
    except Exception:
        english_text = "(Translation failed)"

    # 🔹 Step 2: Model prediction on Arabic text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        suicidal_score = probs[0][1].item()

    # 🔹 Step 3: Decide label
    label = "🟥 Suicidal" if suicidal_score > 0.4 else "🟩 Non-Suicidal"

    return label, f"{suicidal_score:.2f}", text, english_text


# Build a cleaner Gradio UI with multiple output boxes
app = gr.Interface(
    fn=classify_tweet,
    inputs=gr.Textbox(
        lines=3,
        placeholder="✍️ Enter Arabic tweet here... / أدخل التغريدة بالعربية هنا...",
        label="Input Tweet (Arabic)"
    ),
    outputs=[
        gr.Textbox(label="Prediction", interactive=False),
        gr.Textbox(label="Confidence Score", interactive=False),
        gr.Textbox(label="Original Arabic Text", interactive=False),
        gr.Textbox(label="English Translation", interactive=False),
    ],
    title="🌙 Arabic Suicidality Detector",
    description="This app classifies Arabic tweets as **Suicidal** or **Non-Suicidal** and also provides an English translation."
)

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d1ea505960a636eea4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
